In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext autotime

time: 280 µs


In [3]:
%cd ..

/Users/rubenbroekx/Documents/Projects/twitter-sentiment-classifier/twitter_sentiment_classifier
time: 3.22 ms


In [4]:
# Necessary to be in root to get access to store
%cd ..

/Users/rubenbroekx/Documents/Projects/twitter-sentiment-classifier
time: 1.03 ms


# Train SentimentModel

This notebook will train the last `N` layers of the 12-layered RobBERT Transformer model.

In [6]:
import json
import os
import pandas as pd

from twitter_sentiment_classifier.store.loader import fetch_all_tweet_data
from tqdm import tqdm
from collections import Counter

time: 4.53 s


## 1. Load in data

Load in the different datasets (`train`, `test`, and `validation`).

In [7]:
# Fetch all tweet data from S3 (skips automatically if this is already the case)
fetch_all_tweet_data()

time: 1.03 ms


In [8]:
# Training
with open(os.path.expanduser('twitter_sentiment_classifier/store/data/tweets_train.jsonl'), 'r') as f:
    annotations_train = [json.loads(line) for line in f.readlines()]
print(f"Loaded in {len(annotations_train)} training annotations")

# Testing
with open(os.path.expanduser('twitter_sentiment_classifier/store/data/tweets_test.jsonl'), 'r') as f:
    annotations_test = [json.loads(line) for line in f.readlines()]
print(f"Loaded in {len(annotations_test)} testing annotations")

# Validation
with open(os.path.expanduser('twitter_sentiment_classifier/store/data/tweets_val.jsonl'), 'r') as f:
    annotations_val = [json.loads(line) for line in f.readlines()]
print(f"Loaded in {len(annotations_val)} validation annotations")

Loaded in 38723 training annotations
Loaded in 3000 testing annotations
Loaded in 1198 validation annotations
time: 802 ms


## 2. Load model

Load in the model used to train. We choose to train the RobBERT model from scratch, to not introduce any biases in a previously trained model.

In [7]:
from twitter_sentiment_classifier.sentiment_model import SentimentModel

time: 2.14 s


In [8]:
# Load in the model
model = SentimentModel(
    cuda=True,
)
model

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


SentimentModel(
	n_classes=3
	batch_size=16
	use_cuda=True
)

time: 6.97 s


In [9]:
# Start from a clean slate by resetting the model
# model.reset(initialise=False)  # Since destructive effect, comment (just to be sure)

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base a

time: 4.27 s


## 3. Training

Train part of the model for a limited amount of epochs.

In [9]:
# Number of training epochs
N_EPOCHS = 2

# Number of last layers prone to change during training
N_LAYERS = 1

time: 0 ns


In [10]:
model.train(
    n_epochs=N_EPOCHS,
    train_samples=annotations_train,
    val_samples=annotations_val,
    test_samples=annotations_test,
    n_last_layers=N_LAYERS,
)

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,1.048412,0.957215,0.545552,0.530799,0.550853,0.545552
100,0.996829,0.937219,0.548767,0.537993,0.563750,0.548767
150,0.925119,0.901173,0.571275,0.571233,0.574138,0.571275
200,0.899838,0.894577,0.591640,0.581849,0.608303,0.591640
250,0.922927,0.872953,0.604502,0.603887,0.603892,0.604502
300,0.916191,0.873560,0.596999,0.592647,0.598976,0.596999
350,0.867785,0.894419,0.607717,0.602028,0.612981,0.607717
400,0.914740,0.914989,0.575563,0.559207,0.597149,0.575563
450,0.848899,0.885644,0.590568,0.581319,0.593710,0.590568
500,0.898440,0.867119,0.596999,0.596884,0.599212,0.596999


PredictionOutput(predictions=array([[-0.2552457 ,  1.9346485 , -1.5358126 ],
       [-1.329975  ,  1.9941362 , -0.62059444],
       [ 0.74584836, -0.6078937 , -0.19003898],
       ...,
       [-2.1736696 ,  0.00989604,  2.5148888 ],
       [-2.289686  ,  0.19483587,  2.4921966 ],
       [-2.1986108 ,  0.1536    ,  2.3910735 ]], dtype=float32), label_ids=array([1, 1, 0, ..., 2, 2, 2], dtype=int64), metrics={'eval_loss': 0.6768249273300171, 'eval_accuracy': 0.704, 'eval_f1': 0.7029534283867045, 'eval_precision': 0.707343460532036, 'eval_recall': 0.704})

time: 4h 45min 8s


## 4. Evaluation

Evaluate this intermediate version of the model. It should already have a decent performance.

In [18]:
model.eval(annotations_test)

PredictionOutput(predictions=array([[-0.2552457 ,  1.9346485 , -1.5358126 ],
       [-1.329975  ,  1.9941362 , -0.62059444],
       [ 0.74584836, -0.6078937 , -0.19003898],
       ...,
       [-2.1736696 ,  0.00989604,  2.5148888 ],
       [-2.289686  ,  0.19483587,  2.4921966 ],
       [-2.1986108 ,  0.1536    ,  2.3910735 ]], dtype=float32), label_ids=array([1, 1, 0, ..., 2, 2, 2], dtype=int64), metrics={'eval_loss': 0.6768249273300171, 'eval_accuracy': 0.704, 'eval_f1': 0.7029534283867045, 'eval_precision': 0.707343460532036, 'eval_recall': 0.704})

time: 6min 1s
